In [1]:
import os
import sys
import pandas as pd

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))
from database import DB
from analysis.feature import win_percent

In [2]:
db = DB()
db.initialise()

In [3]:
db.get_table_name(view=True)

,view_name
0,view_match_info
1,view_set_match
2,view_set_match_info
3,view_set_match_info_away
4,view_set_match_info_concat
5,view_set_match_info_home
6,view_set_match_player_performance


In [4]:
df = db.get_table('view_set_match_player_performance')

In [5]:
list(df.columns)

['league_name',
 'year',
 'season',
 'date',
 'set_number',
 'team_1',
 'team_2',
 'corresponding_team',
 'tiebreaker',
 'match_round',
 'player_name',
 'champion_name',
 'wdl',
 'side',
 'team_first_blood',
 'team_baron_kills',
 'team_dragon_kills',
 'team_rift_kills',
 'game_length',
 'kills',
 'deaths',
 'assists',
 'double',
 'triple',
 'quadra',
 'penta',
 'first_blood',
 'total_gold',
 'earned_gold',
 'total_cs',
 'minion_kills',
 'monster_kills',
 'cspm',
 'goldat10',
 'golddiffat10',
 'goldat15',
 'golddiffat15',
 'csat10',
 'csdiffat10',
 'csat15',
 'csdiffat15',
 'largest_killing_spree',
 'largest_multi_kill',
 'total_damage_dealt_to_champions',
 'total_damage_taken',
 'vision_wards_bought_in_game']

In [6]:
df.head()

,league_name,year,season,date,set_number,team_1,team_2,corresponding_team,tiebreaker,match_round,...,golddiffat15,csat10,csdiffat10,csat15,csdiffat15,largest_killing_spree,largest_multi_kill,total_damage_dealt_to_champions,total_damage_taken,vision_wards_bought_in_game
0,BRCC,2017,spring,2017-01-24,1,ProGaming Esports,IDM Pirata,ProGaming Esports,0,1,...,2169.0,43.0,4.0,70.0,21.0,NaN,NaN,NaN,NaN,NaN
1,BRCC,2017,spring,2017-01-24,1,ProGaming Esports,IDM Pirata,IDM Pirata,0,1,...,-2169.0,39.0,-4.0,49.0,-21.0,NaN,NaN,NaN,NaN,NaN
2,BRCC,2017,spring,2017-01-24,1,ProGaming Esports,IDM Pirata,IDM Pirata,0,1,...,-1895.0,70.0,-13.0,115.0,-10.0,NaN,NaN,NaN,NaN,NaN
3,BRCC,2017,spring,2017-01-24,1,ProGaming Esports,IDM Pirata,ProGaming Esports,0,1,...,1895.0,83.0,13.0,125.0,10.0,NaN,NaN,NaN,NaN,NaN
4,BRCC,2017,spring,2017-01-24,1,ProGaming Esports,IDM Pirata,IDM Pirata,0,1,...,-2078.0,57.0,-30.0,89.0,-48.0,NaN,NaN,NaN,NaN,NaN


In [70]:
league = 'LCK'
year = 2020
#player_name = 'Ruler'
player_list = ['Ruler', 'Teddy', 'Deft', 'Aiming']
player_team_dict = {'Ruler':'GenG', 'Teddy':'T1', 'Deft':'DragonX', 'Aiming':'KT'}


for player in player_list:
    lck = df[(df['league_name']==league) & (df['year']==year) & (df['player_name']==player)]

    season = 'summer'
    temp = lck[lck['season']==season]

    # 상위권 => T1, Damwon, KT, DragonX
    # 하위권 => Sandbox, Afreeca, SeolHaeOne Prince, Hanwha, Dynamics
    team_list_dict = {'high': ['T1', 'Damwon', 'KT', 'DragonX', 'GenG'], 'low': ['Sandbox', 'Afreeca', 'SeolHaeOne Prince', 'Hanwha', 'Dynamics'], 'all': ['T1', 'Damwon', 'KT', 'DragonX']+['Sandbox', 'Afreeca', 'SeolHaeOne Prince', 'Hanwha', 'Dynamics']}
    player_team = player_team_dict[player]
    print(player_team)
    for team_list in team_list_dict.values():
        try:
            team_list.remove(player_team)
        except:
            pass
        print(team_list)
        temp_df = temp[(temp['team_1'].isin(team_list)) | (temp['team_2'].isin(team_list))]

        temp_df['golddiff'] = temp_df['golddiffat10'].apply(lambda x: 1 if x >= 0 else 0)
        up = temp_df['golddiff'].value_counts()[1]
        down = temp_df['golddiff'].value_counts()[0]

        sumUpDown = temp_df['golddiffat10'].sum()
        print(f'Player: {player}')
        print(f'Team: {team_list}')
        print(f'{up}/{up+down} = {up/(up+down)*100}%')
        print(f'Total sum : {sumUpDown}, Average: {sumUpDown/temp_df.shape[0]}')
        print()
        #print(temp_df)

GenG
['T1', 'Damwon', 'KT', 'DragonX']
Player: Ruler
Team: ['T1', 'Damwon', 'KT', 'DragonX']
12/22 = 54.54545454545454%
Total sum : 4059.0, Average: 184.5

['Sandbox', 'Afreeca', 'SeolHaeOne Prince', 'Hanwha', 'Dynamics']
Player: Ruler
Team: ['Sandbox', 'Afreeca', 'SeolHaeOne Prince', 'Hanwha', 'Dynamics']
15/21 = 71.42857142857143%
Total sum : 6322.0, Average: 301.04761904761904

['T1', 'Damwon', 'KT', 'DragonX', 'Sandbox', 'Afreeca', 'SeolHaeOne Prince', 'Hanwha', 'Dynamics']
Player: Ruler
Team: ['T1', 'Damwon', 'KT', 'DragonX', 'Sandbox', 'Afreeca', 'SeolHaeOne Prince', 'Hanwha', 'Dynamics']
27/43 = 62.7906976744186%
Total sum : 10381.0, Average: 241.41860465116278

T1
['Damwon', 'KT', 'DragonX', 'GenG']
Player: Teddy
Team: ['Damwon', 'KT', 'DragonX', 'GenG']
12/18 = 66.66666666666666%
Total sum : 1188.0, Average: 66.0

['Sandbox', 'Afreeca', 'SeolHaeOne Prince', 'Hanwha', 'Dynamics']
Player: Teddy
Team: ['Sandbox', 'Afreeca', 'SeolHaeOne Prince', 'Hanwha', 'Dynamics']
20/24 = 83.33

C:\Users\jjames\anaconda3\envs\game_data\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [35]:
temp

,league_name,year,season,date,set_number,team_1,team_2,corresponding_team,tiebreaker,match_round,...,golddiffat15,csat10,csdiffat10,csat15,csdiffat15,largest_killing_spree,largest_multi_kill,total_damage_dealt_to_champions,total_damage_taken,vision_wards_bought_in_game
37882,LCK,2020,summer,2020-06-19,1,DragonX,GenG,GenG,0,1,...,-180.0,93.0,0.0,148.0,-8.0,NaN,NaN,NaN,NaN,NaN
37883,LCK,2020,summer,2020-06-19,2,DragonX,GenG,GenG,0,1,...,728.0,79.0,6.0,124.0,18.0,3.0,2.0,11400.0,13400.0,6.0
37907,LCK,2020,summer,2020-06-19,3,DragonX,GenG,GenG,0,1,...,-889.0,83.0,-14.0,140.0,-21.0,NaN,NaN,NaN,NaN,NaN
37991,LCK,2020,summer,2020-06-21,1,KT,GenG,GenG,0,1,...,1421.0,91.0,3.0,157.0,9.0,2.0,1.0,19800.0,15900.0,6.0
37994,LCK,2020,summer,2020-06-21,2,KT,GenG,GenG,0,1,...,1896.0,80.0,76.0,121.0,104.0,NaN,NaN,NaN,NaN,NaN
38016,LCK,2020,summer,2020-06-21,3,KT,GenG,GenG,0,1,...,808.0,86.0,9.0,152.0,16.0,NaN,NaN,NaN,NaN,NaN
38089,LCK,2020,summer,2020-06-25,1,Damwon,GenG,GenG,0,2,...,2764.0,89.0,77.0,140.0,120.0,4.0,3.0,14200.0,11200.0,6.0
38094,LCK,2020,summer,2020-06-25,2,Damwon,GenG,GenG,0,2,...,530.0,99.0,3.0,158.0,12.0,NaN,NaN,NaN,NaN,NaN
38123,LCK,2020,summer,2020-06-25,3,Damwon,GenG,GenG,0,2,...,1563.0,67.0,62.0,116.0,107.0,4.0,2.0,10700.0,9300.0,5.0
38193,LCK,2020,summer,2020-06-27,1,SeolHaeOne Prince,GenG,GenG,0,2,...,483.0,87.0,27.0,139.0,32.0,4.0,2.0,15200.0,12500.0,7.0


In [27]:
temp_df

,league_name,year,season,date,set_number,team_1,team_2,corresponding_team,tiebreaker,match_round,...,csat10,csdiffat10,csat15,csdiffat15,largest_killing_spree,largest_multi_kill,total_damage_dealt_to_champions,total_damage_taken,vision_wards_bought_in_game,golddiff
37882,LCK,2020,summer,2020-06-19,1,DragonX,GenG,GenG,0,1,...,93.0,0.0,148.0,-8.0,NaN,NaN,NaN,NaN,NaN,0
37883,LCK,2020,summer,2020-06-19,2,DragonX,GenG,GenG,0,1,...,79.0,6.0,124.0,18.0,3.0,2.0,11400.0,13400.0,6.0,1
37907,LCK,2020,summer,2020-06-19,3,DragonX,GenG,GenG,0,1,...,83.0,-14.0,140.0,-21.0,NaN,NaN,NaN,NaN,NaN,0
37991,LCK,2020,summer,2020-06-21,1,KT,GenG,GenG,0,1,...,91.0,3.0,157.0,9.0,2.0,1.0,19800.0,15900.0,6.0,1
37994,LCK,2020,summer,2020-06-21,2,KT,GenG,GenG,0,1,...,80.0,76.0,121.0,104.0,NaN,NaN,NaN,NaN,NaN,1
38016,LCK,2020,summer,2020-06-21,3,KT,GenG,GenG,0,1,...,86.0,9.0,152.0,16.0,NaN,NaN,NaN,NaN,NaN,1
38089,LCK,2020,summer,2020-06-25,1,Damwon,GenG,GenG,0,2,...,89.0,77.0,140.0,120.0,4.0,3.0,14200.0,11200.0,6.0,1
38094,LCK,2020,summer,2020-06-25,2,Damwon,GenG,GenG,0,2,...,99.0,3.0,158.0,12.0,NaN,NaN,NaN,NaN,NaN,1
38123,LCK,2020,summer,2020-06-25,3,Damwon,GenG,GenG,0,2,...,67.0,62.0,116.0,107.0,4.0,2.0,10700.0,9300.0,5.0,1
38193,LCK,2020,summer,2020-06-27,1,SeolHaeOne Prince,GenG,GenG,0,2,...,87.0,27.0,139.0,32.0,4.0,2.0,15200.0,12500.0,7.0,1


T1이 매시즌마다 몇경기중에 몇번 이겼는지 구하기

In [7]:
match_df = db.get_table('view_match_info')
win_percent('T1', match_df)

Year: 2017, Season: spring, Win: 16, Loss: 2, Win%: 88.89%
Year: 2017, Season: summer, Win: 13, Loss: 5, Win%: 72.22%
Year: 2018, Season: spring, Win: 9, Loss: 9, Win%: 50.00%
Year: 2018, Season: summer, Win: 8, Loss: 10, Win%: 44.44%
Year: 2019, Season: spring, Win: 14, Loss: 4, Win%: 77.78%
Year: 2019, Season: summer, Win: 11, Loss: 7, Win%: 61.11%
Year: 2020, Season: spring, Win: 14, Loss: 4, Win%: 77.78%
Year: 2020, Season: summer, Win: 13, Loss: 5, Win%: 72.22%


In [11]:
win_percent('KT', match_df)

Year: 2017, Season: spring, Win: 12, Loss: 6, Win%: 66.67%
Year: 2017, Season: summer, Win: 14, Loss: 4, Win%: 77.78%
Year: 2018, Season: spring, Win: 13, Loss: 5, Win%: 72.22%
Year: 2018, Season: summer, Win: 13, Loss: 5, Win%: 72.22%
Year: 2019, Season: spring, Win: 4, Loss: 14, Win%: 22.22%
Year: 2019, Season: summer, Win: 6, Loss: 12, Win%: 33.33%
Year: 2020, Season: spring, Win: 10, Loss: 8, Win%: 55.56%
Year: 2020, Season: summer, Win: 7, Loss: 11, Win%: 38.89%
